# Temporary Notebook: Extracting Initial Game List Data

Scope:
- Games Released from 2018 to 2023
- Non-Indie Games (Games not released on Itch.io)

In [1]:
import pandas as pd
import os
import requests
import numpy as np

# create a .env file and store RAWG_TOKEN="<Your API Key>"
from dotenv import load_dotenv
load_dotenv()

RAWG_TOKEN = os.getenv('RAWG_TOKEN')


from utilities import (
    get_list_response,
    get_detail_response,
    transform_game_list_api,
    transform_game_detail_api,
    create_dictionary_from_df
)

### 1: Inspect number of Requests to send to Game/List

- Unfortunately due to RAWG API's bug we cannot extract more than 10,000 Counts of Data

In [63]:
# pull 1 request to check number of requests needed for game_list
page_1_request = get_list_response(RAWG_TOKEN,
                        "games",
                        page_size=40,
                        exclude_stores="9",
                        dates="2018-01-01,2023-01-31")

In [68]:
print(f"Number of Games to extract: {page_1_request['count']}")
print(f"Number of APIs needed: {int(np.ceil(page_1_request['count'] / 40))}")

Number of Games to extract: 66301
Number of APIs needed: 1658


### 2: Performing Extraction

In [52]:
df_compiled_game_data = pd.DataFrame()
df_compiled_platforms = pd.DataFrame()
df_compiled_stores = pd.DataFrame()
df_compiled_ratings = pd.DataFrame()
df_compiled_status = pd.DataFrame()
df_compiled_tags = pd.DataFrame()
df_compiled_esrb = pd.DataFrame()
df_compiled_parent_platform = pd.DataFrame()
df_compiled_genre = pd.DataFrame()

In [53]:
date_range = [
    "2018-01-01,2018-06-30",
    "2018-07-01,2018-12-31",
    "2019-01-01,2019-06-30",
    "2019-07-01,2019-12-31",
    "2020-01-01,2020-06-30",
    "2020-07-01,2020-12-31",
    "2021-01-01,2021-06-30",
    "2021-07-01,2021-12-31",
    "2022-01-01,2022-06-30",
    "2022-07-01,2022-12-31",
    "2023-01-01,2023-01-31"
]

In [54]:
for range in date_range:
    print(range)
    
    continue_extract = True

    page = 1

    while continue_extract:
        game_list_resp = get_list_response(RAWG_TOKEN, 
                                    "games", 
                                    page_size=40, 
                                    page=page,
                                    exclude_stores="9",
                                    dates=range)
        
        output = transform_game_list_api(game_list_resp["results"])

        df_compiled_game_data = pd.concat([df_compiled_game_data,output["game_data"]])
        df_compiled_platforms = pd.concat([df_compiled_platforms,output["platforms"]])
        df_compiled_stores = pd.concat([df_compiled_stores,output["stores"]])
        df_compiled_ratings = pd.concat([df_compiled_ratings,output["detailed_ratings"]])
        df_compiled_status = pd.concat([df_compiled_status,output["status"]])
        df_compiled_tags = pd.concat([df_compiled_tags,output["tags"]])
        df_compiled_esrb = pd.concat([df_compiled_esrb,output["esrb_rating"]])
        df_compiled_parent_platform = pd.concat([df_compiled_parent_platform,output["parent_platform"]])
        df_compiled_genre = pd.concat([df_compiled_genre,output["genres"]])

        if game_list_resp["next"] != None:
            page += 1
        else:
            continue_extract = False

2018-01-01,2018-06-30
2018-07-01,2018-12-31
2019-01-01,2019-06-30
2019-07-01,2019-12-31
2020-01-01,2020-12-31
2021-01-01,2021-06-30
2021-07-01,2021-12-31
2022-01-01,2022-06-30
2022-07-01,2022-12-31
2023-01-01,2023-01-31


In [65]:
df_compiled_game_data

,slug,name,playtime,released,tba,background_image,rating,rating_top,ratings_count,reviews_text_count,...,suggestions_count,updated,id,score,clip,user_game,reviews_count,saturated_color,dominant_color,community_rating
0,god-of-war-2,God of War (2018),10,2018-04-20,False,https://media.rawg.io/media/games/4be/4be6a6ad...,4.59,5,4255,62,...,580,2023-03-09T23:02:50,58175,None,None,None,4340,0f0f0f,0f0f0f,NaN
1,detroit-become-human,Detroit: Become Human,9,2018-05-25,False,https://media.rawg.io/media/games/951/951572a3...,4.29,5,2787,43,...,463,2023-03-08T08:50:56,29177,None,None,None,2844,0f0f0f,0f0f0f,NaN
2,monster-hunter-world-2,Monster Hunter: World,15,2018-01-26,False,https://media.rawg.io/media/games/21c/21cc15d2...,4.02,4,1493,17,...,571,2023-03-09T21:25:35,46889,None,None,None,1511,0f0f0f,0f0f0f,NaN
3,far-cry-5,Far Cry 5,18,2018-03-27,False,https://media.rawg.io/media/games/bce/bce62fbc...,3.71,4,1862,22,...,374,2023-03-08T08:51:16,23585,None,None,None,1893,0f0f0f,0f0f0f,NaN
4,warhammer-vermintide-2,Warhammer: Vermintide 2,7,2018-03-08,False,https://media.rawg.io/media/games/5be/5bec1462...,3.66,4,655,6,...,246,2023-03-07T20:57:23,45969,None,None,None,664,0f0f0f,0f0f0f,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,modern-cannon-strike,Modern Cannon Strike,0,2023-01-17,False,https://media.rawg.io/media/screenshots/395/39...,0.00,0,0,0,...,0,2023-02-17T09:42:43,926947,None,None,None,0,0f0f0f,0f0f0f,0.0
34,card-brawl-dungeon-slayer,Card Brawl: Dungeon Slayer,0,2023-01-13,False,https://media.rawg.io/media/screenshots/c2e/c2...,0.00,0,0,0,...,0,2023-02-17T08:19:04,926877,None,None,None,0,0f0f0f,0f0f0f,0.0
35,qbert-2023,Q*bert (2023),0,2023-01-24,False,https://media.rawg.io/media/screenshots/f41/f4...,0.00,0,0,0,...,0,2023-02-17T10:09:38,926972,None,None,None,0,0f0f0f,0f0f0f,0.0
36,3d-car-game-parking-jam,3D Car Game: Parking Jam,0,2023-01-27,False,https://media.rawg.io/media/screenshots/3e8/3e...,0.00,0,0,0,...,0,2023-02-17T09:05:19,926914,None,None,None,0,0f0f0f,0f0f0f,0.0


### 3: Export Data

In [58]:
data_directory = os.path.join(os.getcwd(), "initial_upload_raw_data")

In [59]:
df_compiled_game_data.to_csv(os.path.join(data_directory, "game_data.csv"), index=False)
df_compiled_platforms.to_csv(os.path.join(data_directory, "game_platform.csv"), index=False)
df_compiled_stores.to_csv(os.path.join(data_directory, "game_store.csv"), index=False)
df_compiled_ratings.to_csv(os.path.join(data_directory, "game_rating.csv"), index=False)
df_compiled_status.to_csv(os.path.join(data_directory, "game_status.csv"), index=False)
df_compiled_tags.to_csv(os.path.join(data_directory, "game_tag.csv"), index=False)
df_compiled_esrb.to_csv(os.path.join(data_directory, "game_esrb.csv"), index=False)
df_compiled_parent_platform.to_csv(os.path.join(data_directory, "game_parent_platform.csv"), index=False)
df_compiled_genre.to_csv(os.path.join(data_directory, "game_genre.csv"), index=False)